# JSON encoder and decoder
[Internet Data Handling](https://docs.python.org/3/library/json.html)

`json` exposes an API familiar to users of the standard library `marshal` and `pickle` modules.



## Encoding basic Python object hierarchies:

In [1]:
import json

In [2]:
json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])

'["foo", {"bar": ["baz", null, 1.0, 2]}]'

In [3]:
print(json.dumps('\u1234'))

"\u1234"


In [5]:
print(json.dumps('\\'))
print(json.dumps("\"foo\bar"))

"\\"
"\"foo\bar"


In [6]:
print(json.dumps({"c": 0, "b": 0, "a": 0}, sort_keys=True))

{"a": 0, "b": 0, "c": 0}


In [7]:
from io import StringIO

In [8]:
io = StringIO()
json.dump(['streaming API'], io)

In [9]:
io.getvalue()

'["streaming API"]'

In [10]:
# Compact encoding:

json.dumps([1, 2, 3, {'4': 5, '6': 7}], separators=(',', ':'))

'[1,2,3,{"4":5,"6":7}]'

In [11]:
# Вся різниця у відступах ¯\_(ツ)_/¯

json.dumps([1, 2, 3, {'4': 5, '6': 7}])

'[1, 2, 3, {"4": 5, "6": 7}]'

In [20]:
# Pretty printing:

print(json.dumps({'4': 5, '6': 7},
                 sort_keys=True,
                 indent=4))

{
    "4": 5,
    "6": 7
}


## Decoding JSON:

In [21]:
# Тут `null`, в попередньому: `None`

json.loads('["foo", {"bar":["baz", null, 1.0, 2]}]')

['foo', {'bar': ['baz', None, 1.0, 2]}]

In [22]:
json.loads('"\\"foo\\bar"')

'"foo\x08ar'

In [23]:
io = StringIO('["streaming API"]')
json.load(io)

['streaming API']

### Specializing JSON object decoding:

In [24]:
def as_complex(dct):
    if '__complex__' in dct:
        return complex(dct['real'], dct['imag'])
    return dct


json.loads('{"__complex__": true, "real": 1, "imag": 2}',
           object_hook=as_complex)

(1+2j)

In [25]:
import decimal

json.loads('1.1', parse_float=decimal.Decimal)

Decimal('1.1')

In [28]:
complex(1, 1 ** 2)

(1+1j)

### Extending JSONEncoder:

In [29]:
class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, complex):
            return [obj.real, obj.imag]
        # Let the base class default method raise the TypeError
        return json.JSONEncoder.default(self, obj)
    

json.dumps(2 + 1j, cls=ComplexEncoder)

'[2.0, 1.0]'

In [30]:
ComplexEncoder().encode(2 + 1j)

'[2.0, 1.0]'

In [31]:
list(ComplexEncoder().iterencode(2 + 1j))

['[2.0', ', 1.0', ']']

#### default(o)
Implement this method in a subclass such that it returns a serializable object for _o_, or calls the base implementation (to raise a `TypeError`).

For example, to support arbitrary iterators, you could implement default like this:

In [32]:
def default(self, o):
   try:
       iterable = iter(o)
   except TypeError:
       pass
   else:
       return list(iterable)
   # Let the base class default method raise the TypeError
   return json.JSONEncoder.default(self, o)

#### encode(o)
Return a JSON string representation of a Python data structure, _o_. For example:


In [33]:
json.JSONEncoder().encode({"foo": ["bar", "baz"]})

'{"foo": ["bar", "baz"]}'

#### iterencode(o)
Encode the given object, _o_, and yield each string representation as available. 
For example:

In [ ]:
for chunk in json.JSONEncoder().iterencode(bigobject):
    mysocket.write(chunk)

## Infinite and NaN Number Values
The RFC does not permit the representation of infinite or NaN number values. Despite that, by default, this module accepts and outputs `Infinity`, `-Infinity`, and NaN as if they were valid JSON number literal values:


In [34]:
# Neither of these calls raises an exception, 
# but the results are not valid JSON

json.dumps(float('-inf')) 

'-Infinity'

In [35]:
json.dumps(float('nan'))

'NaN'

In [36]:
# Same when deserializing

json.loads('-Infinity')

-inf

In [37]:
json.loads('NaN')

nan

### Repeated Names Within an Object
The RFC specifies that the names within a JSON object should be unique, but does not mandate how repeated names in JSON objects should be handled. By default, this module does not raise an exception; instead, it ignores all but the last name-value pair for a given name:

In [38]:
weird_json = '{"x": 1, "x": 2, "x": 3}'

json.loads(weird_json)

{'x': 3}